To establish communication between Visual Odometry and Kalman filter, homogeneous transformation matrix is needed.
Currently, filters that adopt velocity motion model contains position and angle in x and z-axis to estimate the motion of an agent in a system.

To compute homogeneous transformation matrix, translation vector that contains position in x, y, and z coordinates, and rotation matrix, which can be derived from quaternion, are needed.
In this notebook, mathematical equation that explains new kinematic equation, which conforms to the velocity motion model, is derived.

When converting euler angles from quaternion, gimbal lock, which occurs when pitch angle (angle in y-axis) is +90 or -90 degree, needs to be taken into account.

In [1]:
import numpy as np
import sympy
from sympy.abc import alpha, x, y, v, w, R, psi, theta, phi
from sympy.stats import Normal
from sympy import symbols, Matrix

In [ ]:
# state vector

# position
px = symbols('p_{x}')
py = symbols('p_{y}')
pz = symbols('p_{z}')

# velocity
vx = symbols('v_{x}')
vy = symbols('v_{y}')
vz = symbols('v_{z}')

# quaternion
qw = symbols('q_{w}')
qx = symbols('q_{x}')
qy = symbols('q_{y}')
qz = symbols('q_{z}')

# bias for angular velocity
bwx = symbols('b_{w_x}')
bwy = symbols('b_{w_y}')
bwz = symbols('b_{w_z}')

# bias for linear acceleration
bax = symbols('b_{a_x}')
bay = symbols('b_{a_y}')
baz = symbols('b_{a_z}')

# control input
vf = symbols('v_{f}')

ax = symbols('a_{x}')
ay = symbols('a_{y}')
az = symbols('a_{z}')

# Linear acceleration
a = Matrix([[ax], [ay], [az]])

wx = symbols('w_{x}')
wy = symbols('w_{y}')
wz = symbols('w_{z}')

sigma_w = symbols('sigma_{w}')
sigma_a = symbols('sigma_{a}')

# angular velocity
w = Matrix([[wx], [wy], [wz]])

# delta time
dt = symbols('dt')

# gravitational acceleration
g = Matrix([[0], [0], [-9.81]])

In [3]:
norm_w = symbols('|w|')
R = Matrix([
    [(qw**2 + qx**2 - qy**2 - qz**2), 2*(qx*qy - qw*qz), 2*(qx*qz + qw*qy)],
    [2*(qx*qy + qw*qz), qw**2 - qx**2 + qy**2 - qz**2, 2*(qy*qz - qw*qx)],
    [2*(qx*qz - qw*qy), 2*(qy*qz + qw*qx), qw**2 - qx**2 - qy**2 + qz**2]
])
Omega = Matrix([
    [0, -wx, -wy, -wz],
    [wx, 0, wz, -wy],
    [wy, -wz, 0, wx],
    [wz, wy, -wx, 0]
])

### Basics of Kinematic equation

In [4]:
p = Matrix([[px], [py], [pz]])
v = Matrix([[vx], [vy], [vz]])
q = Matrix([[qw], [qx], [qy], [qz]])
bw = Matrix([[bwx], [bwy], [bwz]])
ba = Matrix([[bax], [bay], [baz]])

ww = Normal("w_{w}", 0, sigma_w)
wa = Normal("w_{a}", 0, sigma_a)

state_x = Matrix([px, py, pz, vx, vy, vz, qw, qx, qy, qz, bwx, bwy, bwz, bax, bay, baz])
control_input = Matrix([ax, ay, az, wx, wy, wz])

In [5]:
a_world = (R @ a + g)

p_t = p + v * dt + 1/2 * a_world * dt**2
v_t = v + a_world * dt

A = sympy.cos(norm_w*dt/2) * sympy.eye(4)
B = (1/norm_w)*sympy.sin(norm_w*dt/2)
q_t = (A + B * Omega) * q

b_w_t = Matrix([
    [bwx + ww],
    [bwy + ww],
    [bwz + ww]
])
b_a_t = Matrix([
    [bax + wa],
    [bay + wa],
    [baz + wa]
])

In [6]:
fxu = Matrix([
    p_t,
    v_t,
    q_t,
    b_w_t,
    b_a_t
])

fxu

Matrix([
[        dt**2*(0.5*a_{x}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2) + 0.5*a_{y}*(-2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + 0.5*a_{z}*(2*q_{w}*q_{y} + 2*q_{x}*q_{z})) + dt*v_{x} + p_{x}],
[        dt**2*(0.5*a_{x}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + 0.5*a_{y}*(q_{w}**2 - q_{x}**2 + q_{y}**2 - q_{z}**2) + 0.5*a_{z}*(-2*q_{w}*q_{x} + 2*q_{y}*q_{z})) + dt*v_{y} + p_{y}],
[dt**2*(0.5*a_{x}*(-2*q_{w}*q_{y} + 2*q_{x}*q_{z}) + 0.5*a_{y}*(2*q_{w}*q_{x} + 2*q_{y}*q_{z}) + 0.5*a_{z}*(q_{w}**2 - q_{x}**2 - q_{y}**2 + q_{z}**2) - 4.905) + dt*v_{z} + p_{z}],
[                                  dt*(a_{x}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2) + a_{y}*(-2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + a_{z}*(2*q_{w}*q_{y} + 2*q_{x}*q_{z})) + v_{x}],
[                                  dt*(a_{x}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + a_{y}*(q_{w}**2 - q_{x}**2 + q_{y}**2 - q_{z}**2) + a_{z}*(-2*q_{w}*q_{x} + 2*q_{y}*q_{z})) + v_{y}],
[                           dt*(a_{x}*(-2*q_{w}*q_{y} + 2*q_{x}*q_{z}) + a_{y}*(2*q_{w

In [7]:
F_kinematics = fxu.jacobian(state_x)
F_kinematics

Matrix([
[1, 0, 0, dt,  0,  0,  dt**2*(1.0*a_{x}*q_{w} - 1.0*a_{y}*q_{z} + 1.0*a_{z}*q_{y}), dt**2*(1.0*a_{x}*q_{x} + 1.0*a_{y}*q_{y} + 1.0*a_{z}*q_{z}), dt**2*(-1.0*a_{x}*q_{y} + 1.0*a_{y}*q_{x} + 1.0*a_{z}*q_{w}), dt**2*(-1.0*a_{x}*q_{z} - 1.0*a_{y}*q_{w} + 1.0*a_{z}*q_{x}), 0, 0, 0, 0, 0, 0],
[0, 1, 0,  0, dt,  0,  dt**2*(1.0*a_{x}*q_{z} + 1.0*a_{y}*q_{w} - 1.0*a_{z}*q_{x}), dt**2*(1.0*a_{x}*q_{y} - 1.0*a_{y}*q_{x} - 1.0*a_{z}*q_{w}),  dt**2*(1.0*a_{x}*q_{x} + 1.0*a_{y}*q_{y} + 1.0*a_{z}*q_{z}),  dt**2*(1.0*a_{x}*q_{w} - 1.0*a_{y}*q_{z} + 1.0*a_{z}*q_{y}), 0, 0, 0, 0, 0, 0],
[0, 0, 1,  0,  0, dt, dt**2*(-1.0*a_{x}*q_{y} + 1.0*a_{y}*q_{x} + 1.0*a_{z}*q_{w}), dt**2*(1.0*a_{x}*q_{z} + 1.0*a_{y}*q_{w} - 1.0*a_{z}*q_{x}), dt**2*(-1.0*a_{x}*q_{w} + 1.0*a_{y}*q_{z} - 1.0*a_{z}*q_{y}),  dt**2*(1.0*a_{x}*q_{x} + 1.0*a_{y}*q_{y} + 1.0*a_{z}*q_{z}), 0, 0, 0, 0, 0, 0],
[0, 0, 0,  1,  0,  0,           dt*(2*a_{x}*q_{w} - 2*a_{y}*q_{z} + 2*a_{z}*q_{y}),          dt*(2*a_{x}*q_{x} + 2*a_{y}*q_{y} 

In [33]:
F_kinematics[0, :]

Matrix([[1, 0, 0, dt, 0, 0, dt**2*(1.0*a_{x}*q_{w} - 1.0*a_{y}*q_{z} + 1.0*a_{z}*q_{y}), dt**2*(1.0*a_{x}*q_{x} + 1.0*a_{y}*q_{y} + 1.0*a_{z}*q_{z}), dt**2*(-1.0*a_{x}*q_{y} + 1.0*a_{y}*q_{x} + 1.0*a_{z}*q_{w}), dt**2*(-1.0*a_{x}*q_{z} - 1.0*a_{y}*q_{w} + 1.0*a_{z}*q_{x}), 0, 0, 0, 0, 0, 0]])

In [9]:
G_kinematics = fxu.jacobian(control_input)
G_kinematics

Matrix([
[dt**2*(0.5*q_{w}**2 + 0.5*q_{x}**2 - 0.5*q_{y}**2 - 0.5*q_{z}**2),                        dt**2*(-1.0*q_{w}*q_{z} + 1.0*q_{x}*q_{y}),                         dt**2*(1.0*q_{w}*q_{y} + 1.0*q_{x}*q_{z}),                        0,                        0,                        0],
[                        dt**2*(1.0*q_{w}*q_{z} + 1.0*q_{x}*q_{y}), dt**2*(0.5*q_{w}**2 - 0.5*q_{x}**2 + 0.5*q_{y}**2 - 0.5*q_{z}**2),                        dt**2*(-1.0*q_{w}*q_{x} + 1.0*q_{y}*q_{z}),                        0,                        0,                        0],
[                       dt**2*(-1.0*q_{w}*q_{y} + 1.0*q_{x}*q_{z}),                         dt**2*(1.0*q_{w}*q_{x} + 1.0*q_{y}*q_{z}), dt**2*(0.5*q_{w}**2 - 0.5*q_{x}**2 - 0.5*q_{y}**2 + 0.5*q_{z}**2),                        0,                        0,                        0],
[                   dt*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2),                               dt*(-2*q_{w}*q_{z} + 2*q_{x}*q_{y}),              

In [10]:
G_kinematics[10, :]

Matrix([[0, 0, 0, 0, 0, 0]])

In [11]:
fxu

Matrix([
[        dt**2*(0.5*a_{x}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2) + 0.5*a_{y}*(-2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + 0.5*a_{z}*(2*q_{w}*q_{y} + 2*q_{x}*q_{z})) + dt*v_{x} + p_{x}],
[        dt**2*(0.5*a_{x}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + 0.5*a_{y}*(q_{w}**2 - q_{x}**2 + q_{y}**2 - q_{z}**2) + 0.5*a_{z}*(-2*q_{w}*q_{x} + 2*q_{y}*q_{z})) + dt*v_{y} + p_{y}],
[dt**2*(0.5*a_{x}*(-2*q_{w}*q_{y} + 2*q_{x}*q_{z}) + 0.5*a_{y}*(2*q_{w}*q_{x} + 2*q_{y}*q_{z}) + 0.5*a_{z}*(q_{w}**2 - q_{x}**2 - q_{y}**2 + q_{z}**2) - 4.905) + dt*v_{z} + p_{z}],
[                                  dt*(a_{x}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2) + a_{y}*(-2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + a_{z}*(2*q_{w}*q_{y} + 2*q_{x}*q_{z})) + v_{x}],
[                                  dt*(a_{x}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + a_{y}*(q_{w}**2 - q_{x}**2 + q_{y}**2 - q_{z}**2) + a_{z}*(-2*q_{w}*q_{x} + 2*q_{y}*q_{z})) + v_{y}],
[                           dt*(a_{x}*(-2*q_{w}*q_{y} + 2*q_{x}*q_{z}) + a_{y}*(2*q_{w

In [12]:
values = {
    px: 0,
    py: 0,
    pz: 0,
    vx: 0,
    vy: 0,
    vz: 0,
    qw: 1,
    qx: 0,
    qy: 0,
    qz: 0,
    bwx: 0.001,
    bwy: 0.001,
    bwz: 0.001,
    bax: 0.001,
    bay: 0.001,
    baz: 0.001,
    ax: 1,
    ay: 1,
    az: 1,
    wx: 1,
    wy: 1,
    wz: 1,
    dt: 0.1,
    norm_w: np.sqrt(1+1+1),
    sigma_w: 0.0001,
    sigma_a: 0.0001,
    ww: 0.0001,
    wa: 0.0001,
}

x_ = np.array(fxu.evalf(subs=values)).astype(float)  # Convert to numpy array and float type
x_

array([[ 0.005     ],
       [ 0.005     ],
       [-0.04405   ],
       [ 0.1       ],
       [ 0.1       ],
       [-0.881     ],
       [ 0.99625234],
       [ 0.04993752],
       [ 0.04993752],
       [ 0.04993752],
       [ 0.0011    ],
       [ 0.0011    ],
       [ 0.0011    ],
       [ 0.0011    ],
       [ 0.0011    ],
       [ 0.0011    ]])

# Velocity model

## Normal condition

In [58]:
A = sympy.cos(norm_w*dt/2) * sympy.eye(4)
B = (1/norm_w)*sympy.sin(norm_w*dt/2)
q_t = (A + B * Omega) * q

_phi = sympy.atan2(2*(qw*qx + qy*qz), qw**2 - qx**2 - qy**2 + qz**2)
_theta = sympy.asin(2*(qw*qy - qx*qz))
_psi = sympy.atan2(2*(qw*qz + qx*qy), qw**2 + qx**2 - qy**2 - qz**2)

rx = vf / wx
rz = vf / wz
dphi = wx * dt
dpsi = wz * dt

In [59]:
fxu_velocity = Matrix([
    [px - rz*sympy.sin(_psi) + rz*sympy.sin(_psi + dpsi)],
    [py + rz*sympy.cos(_psi) - rz*sympy.cos(_psi + dpsi)],
    [pz + rx*sympy.cos(_phi) - rx*sympy.cos(_phi + dphi)],
    v_t,
    q_t,
    b_w_t,
    b_a_t
])
fxu_velocity

Matrix([
[            p_{x} - v_{f}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y})/(w_{z}*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2)) + v_{f}*sin(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/w_{z}],
[p_{y} - v_{f}*cos(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/w_{z} + v_{f}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)/(w_{z}*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2))],
[p_{z} - v_{f}*cos(dt*w_{x} + atan2(2*q_{w}*q_{x} + 2*q_{y}*q_{z}, q_{w}**2 - q_{x}**2 - q_{y}**2 + q_{z}**2))/w_{x} + v_{f}*(q_{w}**2 - q_{x}**2 - q_{y}**2 + q_{z}**2)/(w_{x}*sqrt((2*q_{w}*q_{x} + 2*q_{y}*q_{z})**2 + (q_{w}**2 - q_{x}**2 - q_{y}**2 + q_{z}**2)**2))],
[                                                                                                                          dt*(a_{x}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2) + a_{y}

In [60]:
F_velocity = fxu_velocity.jacobian(state_x)
F_velocity

Matrix([
[1, 0, 0, 0, 0, 0,            -2*q_{z}*v_{f}/(w_{z}*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2)) - v_{f}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y})*(-2*q_{w}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2) - 2*q_{z}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y}))/(w_{z}*((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2)**(3/2)) + v_{f}*(2*q_{w}*(-2*q_{w}*q_{z} - 2*q_{x}*q_{y})/((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2) + 2*q_{z}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)/((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2))*cos(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/w_{z},             -2*q_{y}*v_{f}/(w_{z}*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2)) - v_{f}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y})*(-2*q_{x}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2) - 2*q_{y}*(

In [61]:
F_velocity[10, :]

Matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [62]:
G_velocity = fxu_velocity.jacobian(control_input)
G_velocity

Matrix([
[                                             0,                                              0,                                              0,                                                                                                                                                                                                                                                                                                                                                                                        0,                        0,             dt*v_{f}*cos(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/w_{z} + v_{f}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y})/(w_{z}**2*sqrt((2*q_{w}*q_{z} + 2*q_{x}*q_{y})**2 + (q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2)**2)) - v_{f}*sin(dt*w_{z} + atan2(2*q_{w}*q_{z} + 2*q_{x}*q_{y}, q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2))/w_{z}**2],
[                                             0,               

In [81]:
G_velocity[9, :]

Matrix([[0, 0, 0, -q_{y}*sin(dt*|w|/2)/|w|, q_{x}*sin(dt*|w|/2)/|w|, q_{w}*sin(dt*|w|/2)/|w|]])

In [23]:
values = {
    px: 0,
    py: 0,
    pz: 0,
    vx: 0,
    vy: 0,
    vz: 0,
    qw: 1,
    qx: 0,
    qy: 0,
    qz: 0,
    bwx: 0.001,
    bwy: 0.001,
    bwz: 0.001,
    bax: 0.001,
    bay: 0.001,
    baz: 0.001,
    ax: 1,
    ay: 1,
    az: 1,
    wx: 1,
    wy: 1,
    wz: 1,
    dt: 0.1,
    norm_w: np.sqrt(1+1+1),
    sigma_w: 0.0001,
    sigma_a: 0.0001,
    ww: 0.0001,
    wa: 0.0001,
    vf: np.sqrt(3),
}

x_ = np.array(fxu_velocity.evalf(subs=values)).astype(float)
x_

array([[ 0.17291655],
       [ 0.00865304],
       [ 0.00865304],
       [ 0.1       ],
       [ 0.1       ],
       [-0.881     ],
       [ 0.99625234],
       [ 0.04993752],
       [ 0.04993752],
       [ 0.04993752],
       [ 0.0011    ],
       [ 0.0011    ],
       [ 0.0011    ],
       [ 0.0011    ],
       [ 0.0011    ],
       [ 0.0011    ]])

## when pitch = + pi/2 radian (UP)

https://danceswithcode.net/engineeringnotes/quaternions/quaternions.html

In [82]:
_phi = 0
_theta = sympy.asin(2*(qw*qy - qx*qz))
_psi = -2*sympy.atan2(qx, qw)

fxu_velocity_condition_1 = Matrix([
    [px - rz*sympy.sin(_psi) + rz*sympy.sin(_psi + dpsi)],
    [py + rz*sympy.cos(_psi) - rz*sympy.cos(_psi + dpsi)],
    [pz + rx*sympy.cos(_phi) - rx*sympy.cos(_phi + dphi)],
    v_t,
    q_t,
    b_w_t,
    b_a_t
])
F_velocity_condition_1 = fxu_velocity_condition_1.jacobian(state_x)
G_velocity_condition_1 = fxu_velocity_condition_1.jacobian(control_input)

In [83]:
fxu_velocity_condition_1

Matrix([
[                                                    p_{x} + v_{f}*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} + v_{f}*sin(2*atan2(q_{x}, q_{w}))/w_{z}],
[                                                    p_{y} - v_{f}*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} + v_{f}*cos(2*atan2(q_{x}, q_{w}))/w_{z}],
[                                                                                                       p_{z} - v_{f}*cos(dt*w_{x})/w_{x} + v_{f}/w_{x}],
[       dt*(a_{x}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2) + a_{y}*(-2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + a_{z}*(2*q_{w}*q_{y} + 2*q_{x}*q_{z})) + v_{x}],
[       dt*(a_{x}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + a_{y}*(q_{w}**2 - q_{x}**2 + q_{y}**2 - q_{z}**2) + a_{z}*(-2*q_{w}*q_{x} + 2*q_{y}*q_{z})) + v_{y}],
[dt*(a_{x}*(-2*q_{w}*q_{y} + 2*q_{x}*q_{z}) + a_{y}*(2*q_{w}*q_{x} + 2*q_{y}*q_{z}) + a_{z}*(q_{w}**2 - q_{x}**2 - q_{y}**2 + q_{z}**2) - 9.81) + v_{z}],
[                                   q_{w}*cos(dt*|w|/2) - q_{x}*w_{

In [84]:
F_velocity_condition_1

Matrix([
[1, 0, 0, 0, 0, 0, 2*q_{x}*v_{f}*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) - 2*q_{x}*v_{f}*cos(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)), -2*q_{w}*v_{f}*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) + 2*q_{w}*v_{f}*cos(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)),                                                   0,                                                   0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 2*q_{x}*v_{f}*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) + 2*q_{x}*v_{f}*sin(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)), -2*q_{w}*v_{f}*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) - 2*q_{w}*v_{f}*sin(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)),                                                   0,                                                   0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0,                                                           

In [99]:
F_velocity_condition_1[9, :]

Matrix([[0, 0, 0, 0, 0, 0, w_{z}*sin(dt*|w|/2)/|w|, w_{y}*sin(dt*|w|/2)/|w|, -w_{x}*sin(dt*|w|/2)/|w|, cos(dt*|w|/2), 0, 0, 0, 0, 0, 0]])

In [100]:
G_velocity_condition_1

Matrix([
[                                             0,                                              0,                                              0,                                                                            0,                        0, dt*v_{f}*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} - v_{f}*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z}**2 - v_{f}*sin(2*atan2(q_{x}, q_{w}))/w_{z}**2],
[                                             0,                                              0,                                              0,                                                                            0,                        0, dt*v_{f}*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} + v_{f}*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z}**2 - v_{f}*cos(2*atan2(q_{x}, q_{w}))/w_{z}**2],
[                                             0,                                              0,                                              0, dt*v_{f}*sin(dt*w_{x})/w_{x} + v_{f}*cos

In [111]:
G_velocity_condition_1[9, :]

Matrix([[0, 0, 0, -q_{y}*sin(dt*|w|/2)/|w|, q_{x}*sin(dt*|w|/2)/|w|, q_{w}*sin(dt*|w|/2)/|w|]])

## When pitch = - pi/2 radians (DOWN)

https://danceswithcode.net/engineeringnotes/quaternions/quaternions.html

In [112]:
phi = 0
theta = sympy.asin(2*(qw*qy - qx*qz))
psi = 2*sympy.atan2(qx, qw)
fxu_velocity_condition_2 = Matrix([
    [px - rz*sympy.sin(_psi) + rz*sympy.sin(_psi + dpsi)],
    [py + rz*sympy.cos(_psi) - rz*sympy.cos(_psi + dpsi)],
    [pz + rx*sympy.cos(_phi) - rx*sympy.cos(_phi + dphi)],
    v_t,
    q_t,
    b_w_t,
    b_a_t
])
F_velocity_condition_2 = fxu_velocity_condition_2.jacobian(state_x)
G_velocity_condition_2 = fxu_velocity_condition_2.jacobian(control_input)

In [113]:
fxu_velocity_condition_2

Matrix([
[                                                    p_{x} + v_{f}*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} + v_{f}*sin(2*atan2(q_{x}, q_{w}))/w_{z}],
[                                                    p_{y} - v_{f}*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} + v_{f}*cos(2*atan2(q_{x}, q_{w}))/w_{z}],
[                                                                                                       p_{z} - v_{f}*cos(dt*w_{x})/w_{x} + v_{f}/w_{x}],
[       dt*(a_{x}*(q_{w}**2 + q_{x}**2 - q_{y}**2 - q_{z}**2) + a_{y}*(-2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + a_{z}*(2*q_{w}*q_{y} + 2*q_{x}*q_{z})) + v_{x}],
[       dt*(a_{x}*(2*q_{w}*q_{z} + 2*q_{x}*q_{y}) + a_{y}*(q_{w}**2 - q_{x}**2 + q_{y}**2 - q_{z}**2) + a_{z}*(-2*q_{w}*q_{x} + 2*q_{y}*q_{z})) + v_{y}],
[dt*(a_{x}*(-2*q_{w}*q_{y} + 2*q_{x}*q_{z}) + a_{y}*(2*q_{w}*q_{x} + 2*q_{y}*q_{z}) + a_{z}*(q_{w}**2 - q_{x}**2 - q_{y}**2 + q_{z}**2) - 9.81) + v_{z}],
[                                   q_{w}*cos(dt*|w|/2) - q_{x}*w_{

In [114]:
F_velocity_condition_2

Matrix([
[1, 0, 0, 0, 0, 0, 2*q_{x}*v_{f}*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) - 2*q_{x}*v_{f}*cos(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)), -2*q_{w}*v_{f}*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) + 2*q_{w}*v_{f}*cos(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)),                                                   0,                                                   0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 2*q_{x}*v_{f}*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) + 2*q_{x}*v_{f}*sin(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)), -2*q_{w}*v_{f}*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)) - 2*q_{w}*v_{f}*sin(2*atan2(q_{x}, q_{w}))/(w_{z}*(q_{w}**2 + q_{x}**2)),                                                   0,                                                   0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0,                                                           

In [125]:
F_velocity_condition_2[9, :]

Matrix([[0, 0, 0, 0, 0, 0, w_{z}*sin(dt*|w|/2)/|w|, w_{y}*sin(dt*|w|/2)/|w|, -w_{x}*sin(dt*|w|/2)/|w|, cos(dt*|w|/2), 0, 0, 0, 0, 0, 0]])

In [126]:
G_velocity_condition_2

Matrix([
[                                             0,                                              0,                                              0,                                                                            0,                        0, dt*v_{f}*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} - v_{f}*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z}**2 - v_{f}*sin(2*atan2(q_{x}, q_{w}))/w_{z}**2],
[                                             0,                                              0,                                              0,                                                                            0,                        0, dt*v_{f}*sin(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z} + v_{f}*cos(dt*w_{z} - 2*atan2(q_{x}, q_{w}))/w_{z}**2 - v_{f}*cos(2*atan2(q_{x}, q_{w}))/w_{z}**2],
[                                             0,                                              0,                                              0, dt*v_{f}*sin(dt*w_{x})/w_{x} + v_{f}*cos

In [139]:
G_velocity_condition_2[9, :]

Matrix([[0, 0, 0, -q_{y}*sin(dt*|w|/2)/|w|, q_{x}*sin(dt*|w|/2)/|w|, q_{w}*sin(dt*|w|/2)/|w|]])